In [1]:
import torch

In [2]:
from torch import nn, optim

In [3]:
from torch.utils import data

In [4]:
from loguru import logger

In [5]:
import visualize

In [6]:
import numpy as np

In [7]:
import time

In [8]:
import torchvision
import torchvision.transforms

In [9]:
import resnet, base

In [10]:
def learning_rate_schedules(batch_num, initial_lr, warmup_batchs, total_batchs, decrease_schedules):
    """
    """
    if batch_num < warmup_batchs:
        lr = initial_lr * (batch_num+1)/warmup_batchs
    else:
        num = batch_num - warmup_batchs + 1
        lr = 0.5 * (1 + math.cos(math.pi * (num / total_batchs))) * initial_lr
    return lr

In [11]:
root = 'C:/Datasets/CIFAR10'

In [12]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [30]:
train_transform = torchvision.transforms.Compose(
    [torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
     torchvision.transforms.RandomRotation(10),
     torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomCrop(size=[32,32], padding=4, pad_if_needed=True),
    torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.49140089750289917, 0.4821591377258301, 0.4465310275554657),
                                      (0.24702748656272888, 0.24348321557044983, 0.26158758997917175))
    ])

In [31]:
test_transform= torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.49140089750289917, 0.4821591377258301, 0.4465310275554657),
                                      (0.24702748656272888, 0.24348321557044983, 0.26158758997917175))])

In [32]:
train_dataset = torchvision.datasets.CIFAR10(root, transform=train_transform, download=True)

Files already downloaded and verified


In [33]:
img, label = train_dataset[0]

In [34]:
test_dataset = torchvision.datasets.CIFAR10(root, train=False, transform=test_transform, download=True)

Files already downloaded and verified


In [35]:
train_dt, valid_dt = data.random_split(train_dataset, (40000, 10000))

In [36]:
net = resnet.ResNet32()

2019-06-26 14:33:41.942 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-26 14:33:41.950 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-26 14:33:41.955 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-26 14:33:41.960 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-26 14:33:42.144 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 16, stride: 1
2019-06-26 14:33:42.146 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 16, out_channels: 32, stride: 2
2019-06-26 14:33:42.148 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 32, out_channels: 32, stride: 1
2019-06-26 14:33:42.151 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 32, out_channels: 32, stride: 1
2019-06-26 14:33:42.152 | DEBUG    | resnet:get_residual_unit:158 - in_channels: 32, out_channel

In [38]:
learning_rate = 0.1

In [39]:
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [40]:
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [41]:
batch_size = 512

In [42]:
epoch = 50

In [43]:
use_cuda = net.use_cuda

In [26]:
if use_cuda:
    net = net.cuda()

In [27]:
use_cuda = net.use_cuda

In [28]:
visualizer = visualize.Visualizer('Net')

In [ ]:
for e in range(epoch):
    start_time = time.time()
    net.train()
    train_dt, valid_dt = data.random_split(train_dataset, (40000, 10000))
    train_data_iter = data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    running_loss = 0.0
    for i, (train_data, label) in enumerate(train_data_iter):
        if use_cuda:
            train_data = train_data.cuda()
            label = label.cuda()
        optimizer.zero_grad()
        prediction = net(train_data)
        loss = net.loss(prediction, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if e > 1 and e % 20 == 0:
        learning_rate = learning_rate / 5
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    valid_train_res, valid_ap = base.validate(net, train_dataset, batch_size, num_class=10)
    valid_test_res, test_ap = base.validate(net, test_dataset, batch_size, num_class=10)
    loss_dct = {'loss': running_loss}
    val_precision = valid_train_res['map']
    val_ap_dct = {classes[i]: v for i, v in enumerate(valid_ap)}
    test_precision = valid_test_res['map']
    logger.info(f'epoch num: {e}, loss: {running_loss}, val_precision: {val_precision}, test_precision: {test_precision}, learning_rate: {learning_rate}, time: {time.time() - start_time}')
    visualizer.plot(loss_dct)
    visualizer.plot({'val_precision': val_precision})
    visualizer.plot(val_ap_dct)
    visualizer.plot({'test_precision': test_precision})

2019-06-26 11:11:50.896 | INFO     | __main__:<module>:27 - epoch num: 0, loss: 180.83264923095703, val_precision: 0.17972001433372498, test_precision: 0.1753000020980835, learning_rate: 0.001, time: 74.38428950309753
2019-06-26 11:13:04.575 | INFO     | __main__:<module>:27 - epoch num: 1, loss: 165.21205389499664, val_precision: 0.2555200159549713, test_precision: 0.25529998540878296, learning_rate: 0.001, time: 73.60024309158325
2019-06-26 11:14:22.128 | INFO     | __main__:<module>:27 - epoch num: 2, loss: 146.89814949035645, val_precision: 0.2990799844264984, test_precision: 0.30730003118515015, learning_rate: 0.001, time: 77.48620462417603
2019-06-26 11:15:36.563 | INFO     | __main__:<module>:27 - epoch num: 3, loss: 138.3850153684616, val_precision: 0.33785998821258545, test_precision: 0.33889999985694885, learning_rate: 0.001, time: 74.36424970626831
2019-06-26 11:16:50.567 | INFO     | __main__:<module>:27 - epoch num: 4, loss: 132.1831487417221, val_precision: 0.375359982252

In [149]:
torch.save(net.state_dict(), 'resnet32_01.save')

In [2]:
{'loss': 529.4704940319061, 'val_precision': 0.78054, 'test_precision': 0.6408, 'learning_rate': 0.01, 'time': 19.557708978652954}

{'loss': 529.4704940319061,
 'val_precision': 0.78054,
 'test_precision': 0.6408,
 'learning_rate': 0.01,
 'time': 19.557708978652954}

In [19]:
net = torchvision.models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to C:\Users\emile/.cache\torch\checkpoints\resnet34-333f7ec4.pth
100%|████████████████████████████████████████████████████████████████████| 87306240/87306240 [00:13<00:00, 6268008.65it/s]


In [25]:
fake_input = torch.randn(10, 3, 224, 224)

In [29]:
dir(torchvision.transforms)

['CenterCrop',
 'ColorJitter',
 'Compose',
 'FiveCrop',
 'Grayscale',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'Pad',
 'RandomAffine',
 'RandomApply',
 'RandomChoice',
 'RandomCrop',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomOrder',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'functional',
 'transforms']

In [46]:
import PIL

In [54]:
_filename = 'C:/Users/emile/Pictures/test_0.png'

In [55]:
im = PIL.Image.open(_filename)

In [56]:
im.format

'PNG'

In [57]:
im.size

(500, 282)

In [ ]:
im.